# Particle model for spiral galaxy

In [1]:
import numpy as np

In [78]:
#https://stackoverflow.com/questions/6802577/rotation-of-3d-vector
def rotation_matrix(axis, theta):
    """
    Return the rotation matrix associated with counterclockwise rotation about
    the given axis by theta radians.
    """
    axis = np.asarray(axis)
    axis = axis / np.sqrt(np.dot(axis, axis))
    a = np.cos(theta / 2.0)
    b, c, d = -axis * np.sin(theta / 2.0)
    aa, bb, cc, dd = a * a, b * b, c * c, d * d
    bc, ad, ac, ab, bd, cd = b * c, a * d, a * c, a * b, b * d, c * d
    return np.array([[aa + bb - cc - dd, 2 * (bc + ad), 2 * (bd - ac)],
                     [2 * (bc - ad), aa + cc - bb - dd, 2 * (cd + ab)],
                     [2 * (bd + ac), 2 * (cd - ab), aa + dd - bb - cc]])

def createSpiralGalaxy(arms, starsPerArm, radius, scaleHeight, armThickness, N, B, \
                       armsRot = 0., Nbar = -1, radmax = 1.):
    
    #there must be a better way to do this, but I'm going to save the x,y,z arrays separately and vstack later
    galaxyX = np.array([])
    galaxyY = np.array([])
    galaxyZ = np.array([])

    zaxis = np.array([ 0, 0, 1 ])
    yaxis = np.array([ 0, 1, 0 ])

#http://adsabs.harvard.edu/abs/2009MNRAS.397..164R
    for i in range(starsPerArm):

        angle = (i + 1)/starsPerArm*2.*np.pi
        r = radius/np.log(B*np.tan(angle/(2.*N)))
        if (np.abs(r) < radmax):
            x = r*np.cos(angle) + np.random.normal(loc=0, scale=armThickness)
            y = -r*np.sin(angle) + np.random.normal(loc=0, scale=armThickness)
            z = np.random.normal(loc=0, scale=scaleHeight)

            vect = np.array([x, y, z])
            vect = np.dot(rotation_matrix(zaxis, armsRot), vect)
            vect = np.dot(rotation_matrix(yaxis, np.pi), vect)
            galaxyX = np.append(galaxyX,vect[0])
            galaxyY = np.append(galaxyY,vect[1])
            galaxyZ = np.append(galaxyZ,vect[2])
            
            x = -r*np.cos(angle) + np.random.normal(loc=0, scale=armThickness)
            y = r*np.sin(angle) + np.random.normal(loc=0, scale=armThickness)
            #z = -1.*((np.random.random() - 0.5)*scaleHeight)
            z = np.random.normal(loc=0, scale=scaleHeight)

            vect = np.array([x, y, z])
            vect = np.dot(rotation_matrix(zaxis, armsRot), vect)
            vect = np.dot(rotation_matrix(yaxis, np.pi), vect)
            galaxyX = np.append(galaxyX,vect[0])
            galaxyY = np.append(galaxyY,vect[1])
            galaxyZ = np.append(galaxyZ,vect[2])
            
#connect the bar
    smallAngle = 1./np.abs(starsPerArm)*2.*np.pi
    smallR = radius/np.log(B*np.tan(smallAngle/(2.*N)))
    if (Nbar < 0):
         Nbar = starsPerArm*np.abs(smallR)/radius
    for i in range(int(round(Nbar))):

        r = smallR*(2.*i/Nbar - 1.)
        x = r + np.random.normal(loc=0, scale=armThickness)
        y = -np.random.normal(loc=0, scale=armThickness)
        #z = (np.random.random() - 0.5)*scaleHeight
        z = np.random.normal(loc=0, scale=scaleHeight)

        vect = np.array([x, y, z])
        vect = np.dot(rotation_matrix(zaxis, armsRot), vect)
        vect = np.dot(rotation_matrix(yaxis, np.pi), vect)
        galaxyX = np.append(galaxyX,vect[0])
        galaxyY = np.append(galaxyY,vect[1])
        galaxyZ = np.append(galaxyZ,vect[2])

    return np.vstack((galaxyX, galaxyY, galaxyZ)).T;


In [110]:
radius = 1.2
scaleHeight = 0.15
N = 300. #winding tightness
B = 60. #bulge-to-arm size
armsRot = 0.
arms = 2
starsPerArm = 2000
armThickness = 0.2

gal = createSpiralGalaxy(arms, starsPerArm, radius, scaleHeight, armThickness, N, B, \
                            armsRot = armsRot, Nbar = -1., radmax=100); 


In [83]:
gal

array([[ 0.26617975, -0.08252909,  0.11900837],
       [-0.07990538,  0.12407762, -0.23421829],
       [ 0.13899623, -0.03432531, -0.28208393],
       ...,
       [ 0.29454328, -0.11870372,  0.1867521 ],
       [ 0.18359891,  0.0745802 , -0.13080881],
       [ 0.11253255, -0.10645814, -0.23211261]])

## Look at this in Firefly

In [59]:
# some_file.py
import sys
sys.path.insert(0, '/Users/ageller/Visualizations/Firefly/static/data')

import pandas as pd
from simpleReader import simpleReader
import requests
from IPython.display import IFrame

### Start the server

In [60]:
%%bash --bg --proc bg_proc
python /Users/ageller/Visualizations/Firefly/FireflyFlaskApp.py

### Show Firefly in an Iframe

In [61]:
url = "http://localhost:5000/combined"
IFrame(url, width=1000, height=700)

### Format the data and send it to Firefly

In [111]:
print(gal.shape)

name = "galaxy"
outDict = simpleReader(name, gal, [1,1,0,0.1], 30)
print(outDict['parts'][name]['Coordinates'].shape)

s = pd.Series(outDict).to_json()
requests.post("http://localhost:5000/data_input", json=s)

(4248, 3)
(4248, 3)


<Response [200]>

*To kill the Firefly server*

In [112]:
# uncomment the next line
! ps aux | grep Firefly | awk '{print $2}' | xargs kill

kill: 11584: No such process


## Write to the "raw" file

In [113]:
import csv
csvfile = open("../data/galaxy.raw", 'wt')
csvwriter = csv.writer(csvfile, delimiter=' ', quotechar='|', quoting=csv.QUOTE_MINIMAL)
for i in range(len(gal)):
    csvwriter.writerow([gal[i][0], gal[i][1], gal[i][2], 
                        1, 2, 3, 4, 5, 6])
csvfile.close()